In [1]:
import numpy
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential,Model
import pandas as pd
import pydot
import matplotlib.pyplot as plt
from keras.layers import Input,Flatten,Dense,Lambda,Layer
from keras import backend as K
import numpy as np


In [2]:
(x_train,y_train),(x_test,y_test)=keras.datasets.mnist.load_data()
x_train,x_test=x_train/255.0,x_test/255.0

# Custom Layer

In [16]:
class SimpleDense(Layer):
    #init will initialise the class that accepts the parameters
    #and sets up internal variables
    def __init__(self,units=32):
        super(SimpleDense,self).__init__()
        self.units=units
    #build will run when instance is created. Used to specify 
    #local input states 
    def build(self,input_shape):
        w_init=tf.random_normal_initializer()
        self.w=tf.Variable(name="kernel",
                           initial_value=w_init(shape=(input_shape[-1],self.units),dtype="float32"),
                          trainable=True)
        b_init=tf.zeros_initializer()
        self.b=tf.Variable(name="bias",
                      initial_value=b_init(shape=(self.units,),dtype="float32"),
                      trainable=True)
    def call(self,inputs):
        return tf.matmul(inputs,self.w)+self.b
    

In [17]:
myDense=SimpleDense(units=1)
x=tf.ones((1,1))
y=myDense(x)
print(myDense.variables)

[<tf.Variable 'simple_dense_6/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[0.04396357]], dtype=float32)>, <tf.Variable 'simple_dense_6/bias:0' shape=(1, 1) dtype=float32, numpy=array([[0.]], dtype=float32)>]


# Custom Layer Model

In [18]:
x=np.array([-1.0,0.0,1.0,2.0,3.0,4.0],dtype=float)
y=np.array([-3.0,-1.0,1.0,3.0,5.0,7.0],dtype=float)

In [22]:
myLayer=SimpleDense(units=1)
model=Sequential([myLayer])
model.compile(loss='mse',optimizer='sgd')
model.fit(x,y,epochs=500,verbose=0)

In [23]:
print(model.predict([10.0]))
print(myLayer.variables)

[[18.981436]]
[<tf.Variable 'simple_dense_8/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[1.9973093]], dtype=float32)>, <tf.Variable 'simple_dense_8/bias:0' shape=(1, 1) dtype=float32, numpy=array([[-0.99165815]], dtype=float32)>]


# Custom Layer with Activation

In [28]:
class SimpleDenseActivation(Layer):
    #init will initialise the class that accepts the parameters
    #and sets up internal variables
    def __init__(self,units=32,activation=None):
        super(SimpleDenseActivation,self).__init__()
        self.units=units
        self.activation=tf.keras.activations.get(activation)
    #build will run when instance is created. Used to specify 
    #local input states 
    def build(self,input_shape):
        w_init=tf.random_normal_initializer()
        self.w=tf.Variable(name="kernel",
                           initial_value=w_init(shape=(input_shape[-1],self.units),dtype="float32"),
                          trainable=True)
        b_init=tf.zeros_initializer()
        self.b=tf.Variable(name="bias",
                      initial_value=b_init(shape=(self.units,),dtype="float32"),
                      trainable=True)
#         super().build(input_shape)
    def call(self,inputs):
        return self.activation(tf.matmul(inputs,self.w)+self.b)
    

In [31]:
model=Sequential([
    Flatten(input_shape=[28,28]),
    SimpleDenseActivation(units=128,activation='relu'),
    tf.keras.layers.Dropout(0.2),
    SimpleDenseActivation(units=10,activation='softmax')
])
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit(x_train,y_train,epochs=5)
model.evaluate(x_test,y_test)

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.5446 - accuracy: 0.8516
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1622 - accuracy: 0.9520
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1137 - accuracy: 0.9670
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0947 - accuracy: 0.9714
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.0760 - accuracy: 0.9774


[0.07595747709274292, 0.977400004863739]

In [32]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 784)               0         
_________________________________________________________________
simple_dense_activation_2 (S (None, 128)               100480    
_________________________________________________________________
module_wrapper_2 (ModuleWrap (None, 128)               0         
_________________________________________________________________
simple_dense_activation_3 (S (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________
